<a href="https://colab.research.google.com/github/HimashiRathnayake/Acetobacteraceae_family_phylogenatic_trees/blob/main/STILT_Experiments_Target_Task_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
intermediate_task = "Hi-SA" #@param ["Ru-NLI", "Hi-NLI", "En-NLI", "Hi-SA", "En-SA", "Bn-SA", "Bn-NLI", "Ru-SA", ""]
random_state = 42
intermediate_size = 0 #4000
target_size = 0
dataset_name = "" #@param ["mnli", "cb", "snli", "qnli", ""]
pretrained_model_path = "/content/drive/Shareddrives/Research/Transfer Learning/Experiments/"+ str(random_state) +"/" + intermediate_task + dataset_name + "/" + str(intermediate_size)  + "_3" + "/model" # "xlm-roberta-base" # 
num_labels = 3
output_dir = "/content/drive/Shareddrives/Research/Transfer Learning/Experiments/"+ str(random_state) +"/" + intermediate_task + dataset_name + "-Si-SA/" + str(target_size)  + "_3" + "/"

# Setup

Install Libraries

In [ ]:
# !pip install transformers
# !pip install datasets
# !pip install evaluate
# !pip install sentencepiece

Load Libraries

In [ ]:
import torch
from datasets import load_dataset, load_metric, Dataset, DatasetDict
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Check GPU

In [ ]:
print("GPU Device name")
torch.cuda.get_device_name(0)

GPU Device name


'Tesla T4'

## Load Si-SA

In [ ]:
lankadeepa_data = pd.read_csv("/content/drive/Shareddrives/Research/Transfer Learning/Datasets/Si SA/Si-SA/lankadeepa_tagged_comments.csv")[:9059]
gossipLanka_data = pd.read_csv("/content/drive/Shareddrives/Research/Transfer Learning/Datasets/Si SA/Si-SA/gossip_lanka_tagged_comments.csv")
gossipLanka_data = gossipLanka_data.drop(columns=['Unnamed: 3'])

dataset = pd.concat([lankadeepa_data,gossipLanka_data], ignore_index=True)
indexRemove = dataset[dataset['label'] == 5].index
dataset.drop(indexRemove , inplace=True)
dataset['label'], uniq = pd.factorize(dataset['label'])
dataset['label'].value_counts()
dataset = dataset[['comment', 'label']]
# print((dataset['label'].value_counts()) / len(dataset) * 100)
# np.random.seed(random_state)
# dataset = dataset.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.76055)).reset_index()
# print((dataset['label'].value_counts()) / len(dataset) * 100)
print(len(dataset))
X = dataset['comment'].values.tolist()
y = dataset['label'].values.tolist()

13148


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1000, random_state = random_state)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state = random_state)

In [ ]:
len(X_train), len(X_valid), len(X_test)

(12148, 500, 500)

In [ ]:
np.random.seed(random_state)
train_df = pd.DataFrame({"text": X_train, "label": y_train}).groupby('label', group_keys=False).apply(lambda x: x.sample(frac=12000/len(X_train))).reset_index()
if (target_size) < 12000:
  train_df = train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=10000/len(train_df)))
if (target_size) < 10000:
  train_df = train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=4000/len(train_df)))
if (target_size) < 4000:
  train_df = train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=1000/len(train_df)))
if (target_size) < 1000:
  train_df = train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=500/len(train_df)))
if (target_size) < 500:
  train_df = train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=100/len(train_df)))

valid_df = pd.DataFrame({"text": X_valid, "label": y_valid}).reset_index()
test_df = pd.DataFrame({"text": X_test, "label": y_test}).reset_index()

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
dataset = DatasetDict()
dataset["train"] = train_dataset
dataset["validation"] = valid_dataset
dataset["test"] = test_dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'label', '__index_level_0__'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 500
    })
    test: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 500
    })
})

# Tokenization

In [ ]:
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
train_dataset = tokenized_datasets["train"].shuffle(seed=random_state)
eval_dataset = tokenized_datasets["validation"].shuffle(seed=random_state)
test_dataset = tokenized_datasets["test"]

# Training

In [ ]:
def model_init():
  return XLMRobertaForSequenceClassification.from_pretrained(pretrained_model_path, num_labels=num_labels, ignore_mismatched_sizes=True)

In [ ]:
def compute_metrics(eval_pred):
    accuracy = load_metric("accuracy")
    precision = load_metric("precision")
    recall = load_metric("recall")
    f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = recall.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    return {"accuracy":accuracy, "precision": precision, "recall": recall, "f1": f1}    

In [ ]:
training_args = TrainingArguments(
    num_train_epochs = 3,
    learning_rate = 2e-5,
    output_dir=output_dir, 
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="eval_f1",
    load_best_model_at_end=True,
    seed = random_state,
    data_seed = random_state
)

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

loading configuration file /content/drive/Shareddrives/Research/Transfer Learning/Experiments/42/Hi-SA/0_3/model/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "

In [ ]:
if target_size != 0:
  trainer.train()

In [ ]:
if (target_size != 0):
  trainer.evaluate()

# Generate Predictions


In [ ]:
y_predict = trainer.predict(test_dataset)
preds = np.argmax(y_predict.predictions, axis=-1)
preds

The following columns in the test set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: index, text. If index, text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


In [ ]:
accuracy = load_metric("accuracy")
precision = load_metric("precision")
recall = load_metric("recall")
f1 = load_metric("f1")

accuracy = accuracy.compute(predictions=preds, references=y_test)["accuracy"]
precision = precision.compute(predictions=preds, references=y_test, average="weighted")["precision"]
recall = recall.compute(predictions=preds, references=y_test, average="weighted")["recall"]
f1 = f1.compute(predictions=preds, references=y_test, average="weighted")
{"accuracy":accuracy, "precision": precision, "recall": recall, "f1": f1}  

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

In [ ]:
results = {'Sentence': X_test,
        'Label':y_test,
        'Prediction':preds}
  
# Create DataFrame
df = pd.DataFrame(results)
df.to_csv(output_dir+"/results.csv")

# Save Model

In [ ]:
trainer.save_model(output_dir+"model")